<a href="https://colab.research.google.com/github/RegisKonan/FinalProject/blob/main/Regis_Djaha_GDA_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GDA Implementation.

Implement the Gaussian Discriminant Analysis (GDA) learning algorithm following the steps as discussed in class.

INSTRUCTION: Rename your notebook as: <br>
`firstName_LastName_Live_coding_GDA.ipynb`.

Notes: 
* Do not use any built-in functions to complete a task;
* Do not import additional libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

In [ ]:
# Generate data
def generate_data():
  x, y = make_classification(n_samples= 1000, n_features=3, n_redundant=0, 
                           n_informative=3, random_state=1, 
                           n_clusters_per_class=1)
  
  return x,y

x,y= generate_data() # get data
print(x.shape, y.shape)

(1000, 3) (1000,)


In [ ]:
def split_data(x,y, train_size= 0.8):
    # shuffle the data to randomize the train/test split
    indices = np.random.permutation(len(y))
    
    # Split the shuffled indices into training and testing sets
    train_indices = indices[:int(train_size*len(y))]
    test_indices = indices[int(train_size*len(y)):]
    
    # Use the shuffled indices to extract the corresponding data
    x_train, x_test = x[train_indices], x[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]
    return x_train, x_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test= split_data(x, y, train_size=0.8)# split your data into x_train, x_test, y_train, y_test
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(800, 3) (800,) (200, 3) (200,)


In [ ]:
def covariance(x, mu):
    d = x.shape[1]
    mu=np.mean(x,axis=0)
    cov = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            cov[i, j] = np.sum((x[:,i] - mu[i]) * (x[:,j] - mu[j])) / (len(x) - 1)
    return cov

In [ ]:
print(np.mean(x,axis=0))

[-0.01298493  0.99925415  0.02975341]


In [ ]:
mu=np.mean(x,axis=0)
covariance(x, mu)

array([[1.84495325, 0.02790646, 1.00137533],
       [0.02790646, 1.00170721, 0.05539176],
       [1.00137533, 0.05539176, 1.74832   ]])

In [ ]:
np.cov(x,rowvar=0)

array([[1.84495325, 0.02790646, 1.00137533],
       [0.02790646, 1.00170721, 0.05539176],
       [1.00137533, 0.05539176, 1.74832   ]])

In [ ]:
class GDA:
  def __init__(self):
    ## set mu, phi and sigma to None
    self.mu=None
    self.phi=None
    self.sigma=None
    
  def fit(self,x,y):
    k=2 #np.unique(y).size  # Number of class.
    d=x.shape[1] # input dim
    m= x.shape[0]# Number of examples.
    
    ## Initialize mu, phi and sigma
    self.mu= np.zeros((k,d))#: kxd, i.e., each row contains an individual class mu.
    self.sigma= np.zeros((k,d,d))#: kxdxd, i.e., each row contains an individual class sigma.
    self.phi= np.zeros((k))# d-dimension

    ## START THE LEARNING: estimate mu, phi and sigma.
    for lab in range(k):
        
        self.phi[lab] = np.sum(lab==y)/m 
        self.mu[lab] = np.mean(x[lab==y], axis=0)
        self.sigma[lab] = covariance(x[lab==y], self.mu[lab])
    return self.phi,self.mu, self.sigma
            
            

  def predict_proba(self,x):
    # reshape or flatt x.
    #x= x.reshape(-1, self.mu.shape[1])
    #x=x.reshape(-1,1)
    #x=self.mu.shape[0]
    d= x.shape[0]
    #k_class = self.mu.shape[0] 
    k_class= self.mu.shape[0]  # Number of classes we have in our case it's k = 2
    probabilities = np.zeros((d, k_class))
    det_cov = []
    inv_cov = []
    
    ## START THE LEARNING: estimate mu, phi and sigma.
    for lab in range(k_class):
        det_cov= np.linalg.det(self.sigma[lab])
        inv_cov=np.linalg.inv(self.sigma[lab])
        for j in range(x.shape[0]):
            first_term=1/((2*np.pi)**(d/2)*(det_cov**0.5))
            #first_term=((1/(((2*np.pi)**(d/2))*(det_cov**0.5)))
            exponential=-0.5*((x[j]-self.mu[lab]).T)@(inv_cov)@(x[j]-self.mu[lab])
            second_term=np.exp(exponential)
            probabilities[j, lab] = first_term*second_term*self.phi[lab]
    return probabilities

  def predict(self,x):
    predict=self.predict_proba(x)
    
    y = np.argmax(predict,axis = 1)
    #Predict = np.argmax(self.predict_proba(x))
    return y
    
  
  def accuracy(self, y, ypreds):
#     ypred = self.predict(y)
    result = np.mean(y == ypreds)
    return result * 100 

In [ ]:
model= GDA()
model.fit(X_train,y_train)

(array([0.50125, 0.49875]), array([[ 1.01096213,  0.95110566,  0.99790818],
        [-1.02896802,  0.9313387 , -0.92918824]]), array([[[ 0.85771654, -0.35185093, -0.05609768],
         [-0.35185093,  1.63237487,  0.08749909],
         [-0.05609768,  0.08749909,  0.03753235]],
 
        [[ 0.7646255 ,  0.32103313,  0.14579357],
         [ 0.32103313,  0.34317116, -0.05977341],
         [ 0.14579357, -0.05977341,  1.62657633]]]))

In [ ]:
yproba= model.predict_proba(X_test)
yproba

array([[4.72427614e-081, 9.40365784e-084],
       [4.39885767e-081, 2.16134527e-083],
       [1.88873946e-080, 5.28836763e-084],
       [2.26965753e-080, 1.54819892e-082],
       [2.28091170e-080, 1.00352119e-083],
       [1.30795069e-081, 2.48589677e-082],
       [6.81750560e-230, 1.69129622e-081],
       [5.45315535e-083, 1.22505589e-081],
       [1.70916947e-080, 6.01858641e-083],
       [1.34551669e-080, 4.83688769e-083],
       [7.75507506e-111, 2.18757229e-081],
       [2.37957814e-080, 1.03833446e-082],
       [3.10064531e-080, 1.52060955e-084],
       [5.75740327e-131, 9.96873262e-083],
       [3.12542477e-096, 6.60102747e-081],
       [5.46400922e-081, 4.23910170e-083],
       [1.77370765e-080, 9.04645917e-083],
       [2.86946760e-090, 1.19138943e-080],
       [2.42852668e-081, 1.66204795e-086],
       [2.40959271e-080, 2.76156889e-082],
       [1.02813130e-082, 1.67227568e-100],
       [1.11026040e-239, 7.37482102e-082],
       [9.50266295e-296, 2.03122892e-082],
       [2.3

In [ ]:
ypreds= model.predict(X_test)
ypreds


array([0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0])

In [ ]:
model.accuracy(y_test, ypreds)

97.0

In [ ]:
class LogisticRegression:
  '''
  The goal of this class is to create a LogisticRegression class, 
  that we will use as our model to classify data point into a corresponding class
  '''
  def __init__(self,lr,n_epochs):
    self.lr = lr
    self.n_epochs = n_epochs
    self.train_losses = []
    self.w = None
    self.weight = []

  def add_ones(self, x):
    ##### WRITE YOUR CODE HERE #####
    one = np.ones((x.shape[0],1))
    return np.hstack((one,x))
    #### END CODE ####

  def sigmoid(self, x):
    return 1/(1+np.exp(-x@self.w))


  def cross_entropy(self, x, y_true):
    ##### WRITE YOUR CODE HERE #####
    y_pred = self.sigmoid(x)
    loss = -np.mean(y_true*np.log(y_pred)+(1-y_true)*np.log(1-y_pred))
    return loss
    #### END CODE ####
  
  def predict_proba(self,x):  #This function will use the sigmoid function to compute the probalities
    ##### WRITE YOUR CODE HERE #####
    x= self.add_ones(x)
    proba = self.sigmoid(x)
    return proba
    #### END CODE ####

  def predict(self,x):
    ##### WRITE YOUR CODE HERE #####
    probas = self.predict_proba(x)
    output = [0 if p<0.5 else 1 for p in probas]#np.where(probas>=0.5, 1, 0)      #convert the probalities into 0 and 1 by using a treshold=0.5
    return output
    #### END CODE ####

  def fit(self,x,y):
    # Add ones to x
    x=self.add_ones(x)

    # reshape y if needed
    y=y.reshape(-1,1)

    # Initialize w to zeros vector >>> (x.shape[1])
    self.w=np.zeros((x.shape[1],1))

    for epoch in range(self.n_epochs):
      # make predictions
      ypred = self.sigmoid(x)

      #compute the gradient
      dl = (-1/x.shape[0])*(x.T@(y-ypred))

      #update rule
      self.w=self.w-self.lr*dl

      #Compute and append the training loss in a list
      loss = self.cross_entropy(x,y)
      self.train_losses.append(loss)

      if epoch%1000 == 0:
        print(f'loss for epoch {epoch}  : {loss}')

  def accuracy(self,y_true, y_pred):
    ##### WRITE YOUR CODE HERE #####
    acc = np.mean(y_true==y_pred)*100
    return acc
    #### END CODE ####

In [ ]:
model = LogisticRegression(0.01,n_epochs=10000)
model.fit(X_train,y_train)

loss for epoch 0  : 0.6882420338737469
loss for epoch 1000  : 0.19330810585946673
loss for epoch 2000  : 0.1691979670501055
loss for epoch 3000  : 0.16028077515764566
loss for epoch 4000  : 0.15550339106341185
loss for epoch 5000  : 0.1524625086549569
loss for epoch 6000  : 0.15032926909948766
loss for epoch 7000  : 0.14874049210142104
loss for epoch 8000  : 0.14751029598305337
loss for epoch 9000  : 0.14653223423413902


In [ ]:
ypred_train = model.predict(X_train)
acc = model.accuracy(y_train,ypred_train)
print(f"The training accuracy is: {acc}")
print(" ")

ypred_test = model.predict(X_test)
acc = model.accuracy(y_test,ypred_test)
print(f"The test accuracy is: {acc}")

The training accuracy is: 95.75
 
The test accuracy is: 93.5
